# Flair

В этой части будем решать ту же задачу с тем же датасетом, но теперь попробуем использовать nlp фрейм-ворк flair, который как раз создан для работы с текстами. Установим его:

In [ ]:
!pip install flair -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 25.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 4.2 MB/s eta 0:00:00


Подключимся к гугл-диску:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Импортируем стандартно пандас и нестандартно всё, что нам надо из flair:

In [ ]:
import pandas as pd

from flair.data import Corpus
from flair.datasets import ClassificationCorpus
from flair.embeddings import BytePairEmbeddings
from flair.embeddings import DocumentPoolEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from flair.visual.training_curves import Plotter
from flair.data import Sentence

Создаём директорию для хранения всех нужных файлов:

In [ ]:
import os

directory = '/content/drive/My Drive/colab_data/flair'

if not os.path.exists(directory):
  os.makedirs(directory)

Считываем датасет:

In [ ]:
data = pd.read_csv('/content/drive/My Drive/colab_data/flair/reviews_data.csv')

Чистим, оставляем только самое нужное:

In [ ]:
data.drop('name', axis=1, inplace=True)
data.drop('location', axis=1, inplace=True)
data.drop('Date', axis=1, inplace=True)
data.drop('Image_Links', axis=1, inplace=True)

data = data[data['Review'] != 'No Review Text']
data_cleaned = data.dropna()

data_cleaned.loc[data_cleaned['Rating'] <= 3, 'Rating'] = -1
data_cleaned.loc[data_cleaned['Rating'] >= 4, 'Rating'] = 1
data_cleaned.loc[data_cleaned['Rating'] == -1, 'Rating'] = 0

Приводим все слова к нижнему регистру, убираем лишний мусор:

In [ ]:
data_cleaned.Review = data_cleaned.Review.str.lower()
data_cleaned.Review = data_cleaned.Review.str.replace('[^a-z]+', ' ')
data_cleaned.Review = data_cleaned.Review.str.replace(' +', ' ')
data_cleaned.Review = data_cleaned.Review.str.strip()

<ipython-input-10-6103e047e0a7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned.Review = data_cleaned.Review.str.lower()
<ipython-input-10-6103e047e0a7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned.Review = data_cleaned.Review.str.replace('[^a-z]+', ' ')
<ipython-input-10-6103e047e0a7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [ ]:
data_cleaned.Rating = '__label__' + data_cleaned.Rating.apply(str)

<ipython-input-11-76bd6ab12292>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned.Rating = '__label__' + data_cleaned.Rating.apply(str)


Посмотрим на то, что получилось:

In [ ]:
data_cleaned.head()

,Rating,Review
0,__label__1.0,amber and ladonna at the starbucks on southwes...
1,__label__1.0,** at the starbucks by the fire station on 436...
2,__label__1.0,i just wanted to go out of my way to recognize...
3,__label__1.0,me and my friend were at starbucks and my card...
4,__label__1.0,i’m on this kick of drinking 5 cups of warm wa...


Импортируем numpy, разбиваем выборку на тренировочную, валидационную и тестовую:

In [ ]:
import numpy as np

def train_validate_test_split(df, train_percent=.6, validate_percent=.2, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.loc[perm[:train_end]]
    validate = df.loc[perm[train_end:validate_end]]
    test = df.loc[perm[validate_end:]]
    return train, validate, test

In [ ]:
train, validate, test = train_validate_test_split(data_cleaned, train_percent=.7, validate_percent=.2, seed=700)

In [ ]:
train.shape[0], validate.shape[0], test.shape[0]

(492, 140, 71)

Записываем всё в файлики:

In [ ]:
train.to_csv('/content/drive/My Drive/colab_data/flair/Rating.train.fasttext',
             sep=' ',
             index=False,
             header=None)
validate.to_csv('/content/drive/My Drive/colab_data/flair/Rating.validate.fasttext',
                sep=' ',
                index=False,
                header=None)
test.to_csv('/content/drive/My Drive/colab_data/flair/Rating.test.fasttext',
            sep=' ',
            index=False,
            header=None)

Загрузили...

In [ ]:
data_folder = '/content/drive/My Drive/colab_data/flair/'

corpus: Corpus = ClassificationCorpus(
    data_folder,
    train_file='Rating.train.fasttext',
    dev_file='Rating.validate.fasttext',
    test_file='Rating.test.fasttext')

2024-08-13 07:39:01,028 Reading data from /content/drive/My Drive/colab_data/flair
2024-08-13 07:39:01,030 Train: /content/drive/My Drive/colab_data/flair/Rating.train.fasttext
2024-08-13 07:39:01,033 Dev: /content/drive/My Drive/colab_data/flair/Rating.validate.fasttext
2024-08-13 07:39:01,035 Test: /content/drive/My Drive/colab_data/flair/Rating.test.fasttext
2024-08-13 07:39:01,069 Initialized corpus /content/drive/My Drive/colab_data/flair/ (label type name is 'class')


In [ ]:
print(len(corpus.train), len(corpus.dev), len(corpus.test), sep=', ')

492, 140, 71


Теперь уж точно всё загрузили...

Создаём словарь меток: всё, что может лежать в целевой переменной.

In [ ]:
label_type = 'class'
label_dict = corpus.make_label_dictionary(label_type = label_type)

2024-08-13 09:15:58,116 Computing label dictionary. Progress:


0it [00:00, ?it/s]
492it [00:01, 477.93it/s]

2024-08-13 09:15:59,171 Dictionary created for label 'class' with 2 values: 0.0 (seen 403 times), 1.0 (seen 89 times)


Задачу векторизации будем решать с помощью BPE:

In [ ]:
!pip install flair[word-embeddings] -q

In [ ]:
bpe = BytePairEmbeddings(language='eng', dim=100, syllables=5000)

downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs5000.model


100%|██████████| 315918/315918 [00:00<00:00, 569847.50B/s]


downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs5000.d100.w2v.bin.tar.gz


100%|██████████| 1888515/1888515 [00:00<00:00, 2243965.09B/s]


Теперь нужно решить проблему с разной размерностью. Будем пользоваться классом DocumentPoolEmbeddings, который принимает векторы слов и приводит их к тензору одной размерности - это и есть эмбеддинг всего документа:

In [ ]:
document_emb = DocumentPoolEmbeddings([bpe], fine_tune_mode='nonlinear')

# Обучение - задача bin классификации

Переходим к следующему этапу: созданию классификатора и обучению модели

In [ ]:
classifier = TextClassifier(document_emb, label_dictionary=label_dict, label_type=label_type)

In [ ]:
trainer = ModelTrainer(classifier, corpus)

In [ ]:
import os

model_flair = '/content/drive/My Drive/colab_data/model_flair'

if not os.path.exists(model_flair):
  os.makedirs(model_flair)

In [ ]:
trainer.train(
    model_flair,
    learning_rate=0.1,
    mini_batch_size=64,
    anneal_factor=0.5,
    patience=5,
    max_epochs=2000)

2024-08-13 09:57:50,133 ----------------------------------------------------------------------------------------------------
2024-08-13 09:57:50,141 Model: "TextClassifier(
  (embeddings): DocumentPoolEmbeddings(
    fine_tune_mode=nonlinear, pooling=mean
    (embeddings): StackedEmbeddings(
      (list_embedding_0): BytePairEmbeddings(
        model=0-bpe-eng-5000-100
        (embedding): Embedding(5001, 100)
      )
    )
    (embedding_flex): Linear(in_features=200, out_features=200, bias=False)
    (embedding_flex_nonlinear): ReLU()
    (embedding_flex_nonlinear_map): Linear(in_features=200, out_features=200, bias=True)
  )
  (decoder): Linear(in_features=200, out_features=2, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
  (locked_dropout): LockedDropout(p=0.0)
  (word_dropout): WordDropout(p=0.0)
  (loss_function): CrossEntropyLoss()
  (weights): None
  (weight_tensor) None
)"
2024-08-13 09:57:50,144 -------------------------------------------------------------------------

100%|██████████| 3/3 [00:01<00:00,  2.31it/s]

2024-08-13 09:57:55,981 DEV : loss 0.51125168800354 - f1-score (micro avg)  0.8571


2024-08-13 09:57:57,258  - 0 epochs without improvement
2024-08-13 09:57:57,274  - 0 epochs without improvement
2024-08-13 09:57:57,287 saving best model
2024-08-13 09:57:57,318 ----------------------------------------------------------------------------------------------------
2024-08-13 09:57:57,922 epoch 2 - iter 1/8 - loss 0.52268165 - time (sec): 0.60 - samples/sec: 107.42 - lr: 0.100000 - momentum: 0.000000
2024-08-13 09:57:58,542 epoch 2 - iter 2/8 - loss 0.56202647 - time (sec): 1.22 - samples/sec: 105.23 - lr: 0.100000 - momentum: 0.000000
2024-08-13 09:57:59,204 epoch 2 - iter 3/8 - loss 0.54886254 - time (sec): 1.88 - samples/sec: 102.26 - lr: 0.100000 - momentum: 0.000000
2024-08-13 09:57:59,889 epoch 2 - iter 4/8 - loss 0.51217394 - time (sec): 2.56 - samples/sec: 99.87 - lr: 0.100000 - momentum: 0.000000
2024-08-13 09:58:00,338 epoch 2 - iter 5/8 - loss 0.50786645 - time (sec): 3.01 - samples/sec: 106.26 - lr: 0.100000 - momentum: 0.000000
2024-08-13 09:58:01,504 epoch 2 

100%|██████████| 3/3 [00:00<00:00,  3.21it/s]

...

2024-08-13 10:04:10,140 DEV : loss 0.30890488624572754 - f1-score (micro avg)  0.8857


2024-08-13 10:04:11,091  - 6 epochs without improvement (above 'patience')-> annealing learning_rate to [0.0001953125]
2024-08-13 10:04:11,103  - 6 epochs without improvement (above 'patience')-> annealing learning_rate to [9.765625e-05]
2024-08-13 10:04:11,107 ----------------------------------------------------------------------------------------------------
2024-08-13 10:04:11,116 learning rate too small - quitting training!
2024-08-13 10:04:11,120 ----------------------------------------------------------------------------------------------------
2024-08-13 10:04:11,126 Saving model ...
2024-08-13 10:04:11,159 Done.
2024-08-13 10:04:11,162 ----------------------------------------------------------------------------------------------------
2024-08-13 10:04:11,168 Loading model from best epoch ...


100%|██████████| 2/2 [00:00<00:00,  4.58it/s]

2024-08-13 10:04:11,684 
Results:
- F-score (micro) 0.9014
- F-score (macro) 0.8044
- Accuracy 0.9014

By class:
              precision    recall  f1-score   support

         0.0     0.9048    0.9828    0.9421        58
         1.0     0.8750    0.5385    0.6667        13

    accuracy                         0.9014        71
   macro avg     0.8899    0.7606    0.8044        71
weighted avg     0.8993    0.9014    0.8917        71

2024-08-13 10:04:11,694 ----------------------------------------------------------------------------------------------------


{'test_score': 0.9014084507042254}

Мы добились accuracy 0.9014 в задаче бинарной классификации отзывов на позитивные и негативные, из чего можно сделать вывод, что использование фрейм-ворка Flair является одним из лучших решений.

In [ ]:
my_model = TextClassifier.load('/content/drive/My Drive/colab_data/model_flair/best-model.pt')

Посмотрим, сколько параметров было использовано:

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(count_parameters(my_model))

80602


In [ ]:
import re

def preprocess_text(text):
  text = text.replace('\n', ' ')
  text = re.sub('[^a-zA-Z]+', ' ', text).lower()
  return text.strip()

Давайте на примере отзыва проверим вручную работу модели.

Возьмём позитивный отзыв, обработаем его, и подадим в модель:

In [ ]:
review_1 = """I'm a Starbucks junkie! Absolutely great customer service at each location I've visited. Expensive, but you get what you pay for. Kudos to Mr. Schultz for his continual brilliance."""

In [ ]:
print(preprocess_text(review_1))

i m a starbucks junkie absolutely great customer service at each location i ve visited expensive but you get what you pay for kudos to mr schultz for his continual brilliance


In [ ]:
sentence_1 = Sentence(preprocess_text(review_1))
my_model.predict(sentence_1)

print(sentence_1.labels)

['Sentence[31]: "i m a starbucks junkie absolutely great customer service at each location i ve visited expensive but you get what you pay for kudos to mr schultz for his continual brilliance"'/'1.0' (0.9985)]


Ответ: '1.0' (0.9985)

И это абсолютно правильный ответ, отзыв действительно позитивный

Теперь попробуем сделать то же самое с негативным:

In [ ]:
review_2 = """I was at store #16710 and was waiting, waiting and waiting for my wife's drink and finally I opened my mouth. I called to speak to the manager and she gives the excuse everyone is helping each other. Garbage. They have 3 people on the drive through and one person helping people that walk in. She is a very poor manager, does not know the first thing about customer service or selling. She is poor."""

In [ ]:
print(preprocess_text(review_2))

i was at store and was waiting waiting and waiting for my wife s drink and finally i opened my mouth i called to speak to the manager and she gives the excuse everyone is helping each other garbage they have people on the drive through and one person helping people that walk in she is a very poor manager does not know the first thing about customer service or selling she is poor


In [ ]:
sentence_2 = Sentence(preprocess_text(review_2))
my_model.predict(sentence_2)

print(sentence_2.labels)

['Sentence[74]: "i was at store and was waiting waiting and waiting for my wife s drink and finally i opened my mouth i called to speak to the manager and she gives the excuse everyone is helping each other garbage they have people on the drive through and one person helping people that walk in she is a very poor manager does not know the first thing about customer service or selling she is poor"'/'0.0' (0.9298)]


Ответ: '0.0' (0.9298)

Это также верный ответ, отзыв действительно негативный, и пользователь поставил кофейне оценку 1 из 5.

Таким образом, можем сделать вывод, что использование фрейм-ворка Flair позволяет добиться высокого качества в задаче бинарной классификации текста. Accuracy около 90% получается больше, чем в большинстве остальных методов, поскольку flair предназначен именно для работы с текстом, а до этого мы использовали стандартные методы машинного обучения к тексту, переведённому в векторы.